## Introduction 

### Actions that can be perfomed using this notebook:
- PHOTO
    - take a selfie
    - take a screenshot
    - upload a photo
- VIDEO
    - upload a video
    - do a live video

and then apply a filter 

# **A. Photos**

Sources:
- https://www.digitalocean.com/community/tutorials/how-to-detect-and-extract-faces-from-an-image-with-opencv-and-python  
- https://stackoverflow.com/questions/48512532/cropping-faces-from-an-image-using-opencv-in-python

# 1. Selfie using the computer camera

In [ ]:
# Making necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from utils import *
from tensorflow.keras.models import load_model

In [ ]:
# Loading the model
model_path = "best_model.hdf5"
model = load_model(model_path)

In [ ]:
# Taking the selfie 
image_selfie = camera_grab(camera_id=0)
plt.imshow(image_selfie)
print("dtype: {}, shape: {}, range: {}".format(
    image_selfie.dtype, image_selfie.shape, (image_selfie.min(), image_selfie.max())))

In [ ]:
# Transforming the image 
new_link = 'image_selfie.jpg'
cv2.imwrite('images_source/' + new_link, image_selfie)
image_selfie_output = apply_filters(image_name=new_link,
                                    model=model,
                                    path_to_DL='', 
                                    filter_name='glasses')

In [ ]:
# Displaying the filtered photo 
plt.imshow(image_selfie_output)
plt.show()

# 2. Making a screenshot

In [ ]:
# Making necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from utils import *
from tensorflow.keras.models import load_model

In [ ]:
# Loading the model
model_path = "best_model.hdf5"
model = load_model(model_path)

In [ ]:
# Making a screenshot 
! screencapture -T 3 images_source/timedshot.jpg

In [ ]:
# Adding filters 
image_screen_output = apply_filters(image_name='timedshot.jpg',
                                    model=model,
                                    path_to_DL='', 
                                    filter_name='glasses')

In [ ]:
# Displaying the filtered photo 
plt.imshow(image_screen_output)
plt.show()

# 3. Use a photo of one or many people

In [ ]:
# Making necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from utils import *

In [ ]:
image_to_use = 'a_3_funny1.jpeg'
show_image = plt.imread('images_source/' + image_to_use)
plt.imshow(show_image)
plt.show()

In [ ]:
# Adding filters 
image_screen_output = apply_filters(image_name=image_to_use,
                                    model=model,
                                    path_to_DL='', 
                                    filter_name='glasses')

In [ ]:
# Displaying the filtered photo 
plt.imshow(image_screen_output)
plt.show()

# **B. Videos**

# 1. Live video with a beard filter moved every 5 images

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from utils import *

In [ ]:
# Loading the model
model_path = "best_model.hdf5"
print(f"Model path : {model_path}.")
model = load_model(model_path)
print("Model load was successful!")

In [ ]:
# Filter
santa_filter_original = cv2.imread('filters/santa_filter.png', -1)

In [ ]:
def apply_santa_filter(image, model, rotate_beard_bool=True):
    #santa_filter_original = cv2.imread('filters/santa_filter.png', -1)
    rotate_beard_bool = True
    img_copy = np.copy(image)
    cropped_images, dimensions = crop_image(image)
    counter=0
    
    # Building global variables as we need to reapply the same beard on multiple successive images 
    # for the sake of simplicity, we decided that only one person would appear on the video 
    global x
    global y 
    global right_lip_coords
    global top_lip_coords
    global shift_beard_left
    global shift_beard_top
    global santa_filter
    
    for cropped_image in cropped_images:
        x,y,w,h = dimensions[counter]
        image_test = resize_image(cropped_image) # resize image for the model
        rescaling_factor = cropped_image.shape[0]/96 
        image_model = np.reshape(image_test, (1,96,96,1))/255 # normalise pixel value
        keypoints = model.predict(image_model)[0]

        x_coords = keypoints[0::2]*rescaling_factor
        y_coords = keypoints[1::2]*rescaling_factor
        left_lip_coords = (int(x_coords[11]), int(y_coords[11]))
        
        right_lip_coords = (int(x_coords[12]), int(y_coords[12]))
        top_lip_coords = (int(x_coords[13]), int(y_coords[13]))
        bottom_lip_coords = (int(x_coords[14]), int(y_coords[14]))
        left_eye_coords = (int(x_coords[3]), int(y_coords[3]))
        right_eye_coords = (int(x_coords[5]), int(y_coords[5]))
        brow_coords = (int(x_coords[6]), int(y_coords[6]))

        # Scale filter according to keypoint coordinates
        beard_width = left_lip_coords[0] - right_lip_coords[0]
        beard_length =  int(cropped_image.shape[1]/3*2) # a quarter of the full image
        shift_beard_left =  int(0.25*beard_width)
        shift_beard_top = int(0.1*beard_length)
        scale_beard_factor = 3/2

        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)       # Used for transparency overlay of filter using the alpha channel

        # Beard filter
        santa_filter = cv2.resize(santa_filter_original, (int(beard_width*scale_beard_factor),beard_length))
        sw,sh,sc = santa_filter.shape

        if rotate_beard_bool == True:
            santa_filter = rotate_beard(santa_filter, left_lip_coords, right_lip_coords)

        for i in range(0,sw):   # Overlay the filter based on the alpha channel
            for j in range(0,sh):
                if santa_filter[i,j][3] != 0:
                    try:
                        img_copy[top_lip_coords[1]+y+i-shift_beard_top, right_lip_coords[0]+x+j-shift_beard_left] = santa_filter[i,j][3]
                    except:
                        pass
        counter+=1
    return img_copy

In [ ]:
# Making a video that detects faces and apply the filter every 5 images
video_capture = cv2.VideoCapture(0)
count = 0
ret, image_selfie = video_capture.read()
santa_filter = cv2.imread('filters/santa_filter.png', -1)

while True:
    if (count%5!=0):
        ret, image_selfie = video_capture.read()
        sw,sh,sc = santa_filter.shape
        for i in range(0,sw):   # Overlay the filter based on the alpha channel
            for j in range(0,sh):
                if santa_filter[i,j][3] != 0:
                    try:
                        image_selfie[top_lip_coords[1]+y+i-shift_beard_top, right_lip_coords[0]+x+j-shift_beard_left] = santa_filter[i,j][3]
                    except:
                        pass
        cv2.imshow('Video', image_selfie)
        
    else: 
        # Capture frame-by-frame
        ret, image_selfie = video_capture.read()
        img_copy = apply_santa_filter(image=image_selfie, model=model, rotate_beard_bool=True)

        # Display the resulting frame
        cv2.imshow('Video', img_copy)
    count +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

# 2. Uploading a video and applying filters

Sources: 
- https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames
- https://theailearner.com/2018/10/15/creating-video-from-images-using-opencv-python/

In [1]:
#!pip install ffmpeg-python
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
from utils import *
import ffmpeg

In [2]:
# Filter
santa_filter_original = cv2.imread('filters/santa_filter.png', -1)

In [3]:
def apply_santa_filter(image, model, rotate_beard_bool=True):
    #santa_filter_original = cv2.imread('filters/santa_filter.png', -1)
    rotate_beard_bool = True
    img_copy = np.copy(image)
    cropped_images, dimensions = crop_image(image)
    counter=0
    
    # Building global variables as we need to reapply the same beard on multiple successive images 
    # for the sake of simplicity, we decided that only one person would appear on the video 
    global x
    global y 
    global right_lip_coords
    global top_lip_coords
    global shift_beard_left
    global shift_beard_top
    global santa_filter
    
    for cropped_image in cropped_images:
        x,y,w,h = dimensions[counter]
        image_test = resize_image(cropped_image) # resize image for the model
        rescaling_factor = cropped_image.shape[0]/96 
        image_model = np.reshape(image_test, (1,96,96,1))/255 # normalise pixel value
        keypoints = model.predict(image_model)[0]

        x_coords = keypoints[0::2]*rescaling_factor
        y_coords = keypoints[1::2]*rescaling_factor
        left_lip_coords = (int(x_coords[11]), int(y_coords[11]))
        
        right_lip_coords = (int(x_coords[12]), int(y_coords[12]))
        top_lip_coords = (int(x_coords[13]), int(y_coords[13]))
        bottom_lip_coords = (int(x_coords[14]), int(y_coords[14]))
        left_eye_coords = (int(x_coords[3]), int(y_coords[3]))
        right_eye_coords = (int(x_coords[5]), int(y_coords[5]))
        brow_coords = (int(x_coords[6]), int(y_coords[6]))

        # Scale filter according to keypoint coordinates
        beard_width = left_lip_coords[0] - right_lip_coords[0]
        beard_length =  int(cropped_image.shape[1]/3*2) # a quarter of the full image
        shift_beard_left =  int(0.25*beard_width)
        shift_beard_top = int(0.1*beard_length)
        scale_beard_factor = 3/2

        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)       # Used for transparency overlay of filter using the alpha channel

        # Beard filter
        santa_filter = cv2.resize(santa_filter_original, (int(beard_width*scale_beard_factor),beard_length))
        sw,sh,sc = santa_filter.shape

        if rotate_beard_bool == True:
            santa_filter = rotate_beard(santa_filter, left_lip_coords, right_lip_coords)

        for i in range(0,sw):   # Overlay the filter based on the alpha channel
            for j in range(0,sh):
                if santa_filter[i,j][3] != 0:
                    try:
                        img_copy[top_lip_coords[1]+y+i-shift_beard_top, right_lip_coords[0]+x+j-shift_beard_left] = santa_filter[i,j][3]
                    except:
                        pass
        counter+=1
    return img_copy

In [4]:
# Enter name of the video to use and the name that you want to give to the filtered output
video_input_name = 'pantin_vic.mov'
video_output_name = 'output_pantin_2.mov'
video_gif_name = 'pantin_vic.gif'

In [5]:
# Saving all images from the video in the right order 
vidcap = cv2.VideoCapture('videos_source/' + video_input_name)
success,image = vidcap.read()
count = 0
list_images = []
while success:
    if count < 10:
        count_name = '00' + str(count)
    elif count < 100:
        count_name = '0' + str(count)
    else:
        count_name = str(count)
    cv2.imwrite("videos_source/pantin_vic_images/frame" + count_name + ".jpg", image)     # save frame as JPEG file      
    success,image = vidcap.read()
    list_images.append("videos_source/pantin_vic_images/frame" + count_name + ".jpg") # the ordered list of all images 
    count += 1
print('It was a success')

It was a success


In [6]:
# Loading the model 
model_path = "best_model.hdf5"
model = load_model(model_path)

In [7]:
# Function to apply a filter on each image 
def get_transform(image_name, apply_both=False):
    image_selfie = plt.imread(image_name)
    if apply_both == False:
        return apply_santa_filter(image_selfie, model)
    else:
        return apply_both_filters(image_selfie, model)

In [8]:
# Applying a filter on all images 
for img in list_images:
    image = get_transform(img)
    cv2.imwrite(img, image)

In [15]:
# Making the new video from all images with a filter 
(
    ffmpeg
    .input('videos_source/pantin_vic_images/*.jpg', pattern_type='glob', framerate=20)
    .output(video_output_name)
    .run()
)

(None, None)

# **C. Extra - Creating GIFs**

In [ ]:
#!pip install ez_setup
#!pip install moviepy

In [16]:
from moviepy.editor import *
clip = (VideoFileClip(video_output_name)
        .subclip((0,2.65),(0,3.2))
        .resize(0.3))
clip.write_gif('results/' + video_gif_name)

t:  25%|██▌       | 3/12 [00:00<00:00, 17.18it/s, now=None]

MoviePy - Building file results/pantin_vic.gif with imageio.
